In [1]:
import numpy as np
import matplotlib.pyplot as plt

data    = np.genfromtxt("data-nonlinear.txt", delimiter=',')

class Model:
    def __init__(self):
        self.weight = np.zeros(16)
        self.bias = np.zeros(1)
        
    def __call__(self, x: np.ndarray):
        x = (self.weight * x).sum(axis=1, keepdims=True) + self.bias
        x = 1 / (1 + np.exp(-x)) 
        return x

def BCELoss(pred: np.ndarray, label: np.ndarray):
    return -(label * np.log(pred) + (1 - label) * np.log(1 - pred)).mean(dtype=np.float64)

In [2]:
x_train = data[:, 0:2]
y_train = data[:, 2:3]

transformed_data = x_train
for i in range(2, 7):
    transformed_data = np.concatenate((transformed_data, x_train ** i), axis=1)

    
for i in range(2):
    for j in range(2):
        transformed_data = np.concatenate((transformed_data, x_train[:, 0:1] ** i * x_train[:, 1:2] ** j), axis=1)

transformed_data.shape

(118, 16)